<a href="https://colab.research.google.com/github/vadigr123/colab_telegrams/blob/main/safetensors2diffusers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Installing
!apt -y install -qq aria2
!pip install -q diffusers transformers accelerate safetensors huggingface_hub omegaconf

In [ ]:
#@title Download Checkpoints (SDXL/SD Models) - Standalone { display-mode: "form" }
#@markdown Download SDXL and SD 1.5 checkpoint models from various sources

import os
import locale
locale.getpreferredencoding = lambda: "UTF-8"
%cd /content

def download_things(directory, url, hf_token="", civitai_api_key=""):
    """Download function for various sources"""
    url = url.strip()

    if "drive.google.com" in url:
        original_dir = os.getcwd()
        os.chdir(directory)
        !gdown --fuzzy {url}
        os.chdir(original_dir)
    elif "huggingface.co" in url:
        url = url.replace("?download=true", "")
        if "/blob/" in url:
            url = url.replace("/blob/", "/resolve/")
        user_header = f'"Authorization: Bearer {hf_token}"'
        !aria2c --console-log-level=error --summary-interval=10 --header={user_header} -c -x 16 -k 1M -s 16 {url} -d {directory}  -o {url.split('/')[-1]}
    elif "civitai.com" in url:
        if civitai_api_key:
            # Add the API key as a parameter
            separator = "&" if "?" in url else "?"
            final_url = f"{url}{separator}token={civitai_api_key}"
            !aria2c --console-log-level=error --summary-interval=10 -c -x 16 -k 1M -s 16 -d {directory} "{final_url}"
        else:
            print("⚠️ You need an API key to download Civitai models.")
            return False
    else:
        !aria2c --console-log-level=error --summary-interval=10 -c -x 16 -k 1M -s 16 -d {directory} {url}

    return True

# Create models directory
directory_models = 'models'
os.makedirs(directory_models, exist_ok=True)

#@markdown ---
#@markdown **Model URLs** (comma-separated): Civitai API, Google Drive, or Hugging Face links
checkpoint_urls = "" #@param {type:"string"}

#@markdown ---
#@markdown **API Keys** (if needed)
#@markdown Civitai API Key (required for Civitai downloads) - Get from https://civitai.com/user/account
civitai_key = ""  #@param {type:"string"}
#@markdown Hugging Face Token (for private repos)
hf_token = ""  #@param {type:"string"}

#@markdown ---
def get_existing_models():
    """Get list of existing model files"""
    valid_extensions = {'.ckpt', '.pt', '.pth', '.safetensors', '.bin'}
    existing_files = []

    if os.path.exists(directory_models):
        for filename in os.listdir(directory_models):
            if any(filename.endswith(ext) for ext in valid_extensions):
                existing_files.append(filename)

    return existing_files

def download_checkpoints(urls_string, civitai_api_key="", hf_token=""):
    """Download checkpoint models from provided URLs"""
    if not urls_string.strip():
        print("⚠️ No URLs provided!")
        return

    urls = [url.strip() for url in urls_string.split(',') if url.strip()]

    if not urls:
        print("⚠️ No valid URLs found!")
        return

    # Check for Civitai URLs without API key
    civitai_urls = [url for url in urls if "civitai.com" in url]
    if civitai_urls and not civitai_api_key.strip():
        print("🔑 Civitai API Key Required!")
        print("   You have Civitai URLs but no API key provided.")
        print("   Get your API key from: https://civitai.com/user/account")
        print("   Then add it to the 'civitai_key' field above.")
        return

    print(f"📥 Starting download of {len(urls)} model(s)...")
    print("=" * 50)

    existing_before = get_existing_models()
    downloaded_count = 0
    failed_count = 0

    for i, url in enumerate(urls, 1):
        print(f"🔄 [{i}/{len(urls)}] Processing: {url}")

        # For debugging, show the final URL for Civitai
        if "civitai.com" in url and civitai_api_key:
            separator = "&" if "?" in url else "?"
            final_url = f"{url}{separator}token={civitai_api_key}"
            print(f"   🔗 Final URL: {final_url[:80]}...")

        try:
            success = download_things(directory_models, url, hf_token, civitai_api_key)
            if success is False:
                failed_count += 1
                print(f"   ❌ Download failed!")
            else:
                downloaded_count += 1
                print(f"   ✅ Download completed!")
        except Exception as e:
            failed_count += 1
            print(f"   ❌ Error: {str(e)}")

        print("-" * 30)

    # Check what was actually downloaded
    existing_after = get_existing_models()
    new_files = set(existing_after) - set(existing_before)

    print("=" * 50)
    print(f"📊 Download Summary:")
    print(f"   ✅ Successful downloads: {downloaded_count}")
    print(f"   ❌ Failed downloads: {failed_count}")
    print(f"   📁 New files added: {len(new_files)}")

    if new_files:
        print(f"\n🆕 New model files:")
        for file in sorted(new_files):
            file_path = os.path.join(directory_models, file)
            if os.path.exists(file_path):
                file_size = os.path.getsize(file_path) / (1024**3)  # Size in GB
                print(f"   📄 {file} ({file_size:.2f} GB)")

# Execute the download
print("🚀 Checkpoint Downloader Starting...")
print("=" * 50)

# Verify API key is present
if civitai_key.strip():
    print(f"🔑 Civitai API Key: {civitai_key[:10]}...")
else:
    print("⚠️ No Civitai API Key provided")

try:
    download_checkpoints(checkpoint_urls, civitai_key, hf_token)

    # List all available models
    print("\n🗂️ All Available Models:")
    print("=" * 40)

    all_models = get_existing_models()

    if all_models:
        total_size = 0
        for filename in sorted(all_models):
            file_path = os.path.join(directory_models, filename)
            if os.path.exists(file_path):
                file_size = os.path.getsize(file_path) / (1024**3)  # Size in GB
                total_size += file_size
                print(f"📄 {filename} ({file_size:.2f} GB)")

        print(f"\n💾 Total storage used: {total_size:.2f} GB")
    else:
        print("   No models found in the models directory.")

except Exception as e:
    print(f"❌ Unexpected error: {str(e)}")

print("\n🎉 Checkpoint download process completed!")

In [ ]:
# SafeTensor to Diffusers Converter for Google Colab
# This script converts SafeTensor checkpoint models to Diffusers format and uploads to Hugging Face

# Install required packages
!pip install -q diffusers transformers accelerate safetensors huggingface_hub omegaconf

import os
import json
import torch
from pathlib import Path
from safetensors.torch import load_file
from diffusers import StableDiffusionPipeline, DiffusionPipeline
from huggingface_hub import HfApi, create_repo, upload_folder
from transformers import CLIPTextModel, CLIPTokenizer
import shutil
import tempfile
from typing import Optional, Dict, Any

class SafeTensorToDiffusersConverter:
    def __init__(self):
        self.supported_base_models = {
            'sd1.5': 'runwayml/stable-diffusion-v1-5',
            'sd2.1': 'stabilityai/stable-diffusion-2-1',
            'sdxl': 'stabilityai/stable-diffusion-xl-base-1.0',
            'sd1.4': 'CompVis/stable-diffusion-v1-4',
            'sd2.0': 'stabilityai/stable-diffusion-2',
            'pony': 'AstraliteHeart/pony-diffusion-v6-xl',
            'noobai': 'Laxhar/noobai-XL-1.0',
            'pony_v6': 'AstraliteHeart/pony-diffusion-v6-xl',
            'noobai_xl': 'Laxhar/noobai-XL-1.0',
        }

    def detect_base_model(self, checkpoint_path: str) -> str:
        """
        Detect the base model from SafeTensor checkpoint
        """
        try:
            # Load the safetensor file
            state_dict = load_file(checkpoint_path, device="cpu")

            # Check for specific model signatures
            filename = os.path.basename(checkpoint_path).lower()

            # Check filename for model type hints
            if any(term in filename for term in ['pony', 'pdxl', 'ponyxl']):
                return self.supported_base_models['pony']
            elif any(term in filename for term in ['noob', 'nai', 'noobai']):
                return self.supported_base_models['noobai']

            # Check model dimensions and architecture to determine base model
            # Look for specific keys that indicate model type
            if "model.diffusion_model.input_blocks.0.0.weight" in state_dict:
                input_channels = state_dict["model.diffusion_model.input_blocks.0.0.weight"].shape[1]

                # Check UNet architecture
                if "model.diffusion_model.middle_block.1.proj_in.weight" in state_dict:
                    middle_dim = state_dict["model.diffusion_model.middle_block.1.proj_in.weight"].shape[0]
                    if middle_dim == 2048:  # SDXL-based (could be Pony or NoobAI)
                        # Additional checks for Pony/NoobAI specific features
                        if self._is_pony_model(state_dict):
                            return self.supported_base_models['pony']
                        elif self._is_noobai_model(state_dict):
                            return self.supported_base_models['noobai']
                        else:
                            return self.supported_base_models['sdxl']
                    elif middle_dim == 1024:  # SD 2.x
                        return self.supported_base_models['sd2.1']
                    else:  # SD 1.x
                        return self.supported_base_models['sd1.5']

            # Fallback: check file size as rough indicator
            file_size = os.path.getsize(checkpoint_path) / (1024**3)  # GB
            if file_size > 5:  # SDXL-based models are typically larger
                return self.supported_base_models['sdxl']
            elif file_size > 3:  # SD 2.x models
                return self.supported_base_models['sd2.1']
            else:  # SD 1.x models
                return self.supported_base_models['sd1.5']

        except Exception as e:
            print(f"Error detecting base model: {e}")
            print("Defaulting to Stable Diffusion XL")
            return self.supported_base_models['sdxl']

    def _is_pony_model(self, state_dict: Dict[str, Any]) -> bool:
        """Check if the model has Pony-specific characteristics"""
        # Pony models often have specific layer configurations
        # This is a heuristic check - you might need to adjust based on actual Pony model structure
        pony_indicators = [
            "model.diffusion_model.label_emb",  # Pony often has label embedding
            "model.diffusion_model.time_embed.2",  # Specific time embedding structure
        ]
        return any(key in state_dict for key in pony_indicators)

    def _is_noobai_model(self, state_dict: Dict[str, Any]) -> bool:
        """Check if the model has NoobAI-specific characteristics"""
        # NoobAI models might have specific architectural features
        # This is a heuristic check - adjust based on actual NoobAI model structure
        noobai_indicators = [
            # Add specific NoobAI model indicators here
            "model.diffusion_model.zero_convs",  # Example indicator
        ]
        return any(key in state_dict for key in noobai_indicators)

    def convert_safetensor_to_diffusers(self, checkpoint_path: str, output_dir: str, base_model: str = None) -> str:
        """
        Convert SafeTensor checkpoint to Diffusers format
        """
        try:
            # Detect base model if not provided
            if base_model is None:
                base_model = self.detect_base_model(checkpoint_path)
                print(f"Detected base model: {base_model}")

            # Create temporary directory for conversion
            temp_dir = tempfile.mkdtemp()

            # Load the base pipeline
            print(f"Loading base pipeline from {base_model}...")
            if "xl" in base_model.lower():
                from diffusers import StableDiffusionXLPipeline
                pipeline = StableDiffusionXLPipeline.from_pretrained(
                    base_model,
                    torch_dtype=torch.float16,
                    use_safetensors=True
                )
            else:
                pipeline = StableDiffusionPipeline.from_pretrained(
                    base_model,
                    torch_dtype=torch.float16,
                    use_safetensors=True
                )

            # Load checkpoint weights
            print("Loading checkpoint weights...")
            checkpoint = load_file(checkpoint_path, device="cpu")

            # Convert checkpoint format to diffusers format
            converted_state_dict = self._convert_checkpoint_to_diffusers(checkpoint, pipeline)

            # Load converted weights into pipeline
            if converted_state_dict:
                pipeline.unet.load_state_dict(converted_state_dict.get('unet', {}), strict=False)
                if 'vae' in converted_state_dict:
                    pipeline.vae.load_state_dict(converted_state_dict['vae'], strict=False)
                if 'text_encoder' in converted_state_dict:
                    pipeline.text_encoder.load_state_dict(converted_state_dict['text_encoder'], strict=False)

            # Save the converted pipeline
            print(f"Saving converted pipeline to {output_dir}...")
            os.makedirs(output_dir, exist_ok=True)
            pipeline.save_pretrained(output_dir, safe_serialization=True)

            # Create model card - Fixed method name
            self._create_detailed_model_card(output_dir, base_model, checkpoint_path)

            print("Conversion completed successfully!")
            return output_dir

        except Exception as e:
            print(f"Error during conversion: {e}")
            raise e

    def _convert_checkpoint_to_diffusers(self, checkpoint: Dict[str, Any], pipeline) -> Dict[str, Any]:
        """
        Convert checkpoint state dict to diffusers format
        """
        converted = {}

        # This is a simplified conversion - in practice, you might need more sophisticated mapping
        # depending on the specific checkpoint format

        unet_state_dict = {}
        vae_state_dict = {}
        text_encoder_state_dict = {}

        for key, value in checkpoint.items():
            # Map UNet weights
            if key.startswith('model.diffusion_model.'):
                new_key = key.replace('model.diffusion_model.', '')
                unet_state_dict[new_key] = value

            # Map VAE weights
            elif key.startswith('first_stage_model.'):
                new_key = key.replace('first_stage_model.', '')
                vae_state_dict[new_key] = value

            # Map Text Encoder weights
            elif key.startswith('cond_stage_model.'):
                new_key = key.replace('cond_stage_model.', '')
                text_encoder_state_dict[new_key] = value

        if unet_state_dict:
            converted['unet'] = unet_state_dict
        if vae_state_dict:
            converted['vae'] = vae_state_dict
        if text_encoder_state_dict:
            converted['text_encoder'] = text_encoder_state_dict

        return converted

    def _create_detailed_model_card(self, output_dir: str, base_model: str, checkpoint_path: str, model_name: str = "", model_description: str = ""):
        """
        Create a detailed model card for the converted model
        """
        if not model_name:
            model_name = os.path.basename(checkpoint_path).replace('.safetensors', '').replace('.ckpt', '')

        # Determine model type for tags
        model_type = "sdxl" if any(term in base_model.lower() for term in ['xl', 'pony', 'noob']) else "sd"

        model_card_content = f"""---
license: creativeml-openrail-m
base_model: {base_model}
tags:
- stable-diffusion
- stable-diffusion-diffusers
- stable-diffusion-{model_type}
- text-to-image
- diffusers
- safetensors
{"- pony" if "pony" in base_model.lower() else ""}
{"- noobai" if "noob" in base_model.lower() else ""}
widget:
- text: "a beautiful landscape, masterpiece, best quality"
  example_title: "Landscape"
- text: "1girl, anime style, detailed face, high quality"
  example_title: "Anime Character"
- text: "a cute cat, photorealistic, 4k"
  example_title: "Photorealistic"
inference: true
---

# {model_name}

{model_description}

## Model Information

- **Base Model**: {base_model}
- **Original Format**: SafeTensor Checkpoint
- **Converted Format**: Diffusers
- **Model Type**: {"Stable Diffusion XL" if "xl" in model_type else "Stable Diffusion"}
- **Original Checkpoint**: {os.path.basename(checkpoint_path)}

## Usage

### Basic Usage

```python
from diffusers import {"StableDiffusionXLPipeline" if model_type == "sdxl" else "StableDiffusionPipeline"}
import torch

# Load the pipeline
pipeline = {"StableDiffusionXLPipeline" if model_type == "sdxl" else "StableDiffusionPipeline"}.from_pretrained(
    "your-username/your-repo-name",
    torch_dtype=torch.float16,
    use_safetensors=True
)
pipeline = pipeline.to("cuda")

# Generate an image
prompt = "a beautiful landscape, masterpiece, best quality"
{"negative_prompt = 'low quality, blurry, distorted'" if model_type == "sdxl" else ""}
image = pipeline(
    prompt,
    {"negative_prompt=negative_prompt," if model_type == "sdxl" else ""}
    num_inference_steps=25,
    guidance_scale=7.5,
    {"width=1024, height=1024" if model_type == "sdxl" else "width=512, height=512"}
).images[0]

image.save("generated_image.png")
```

### Advanced Usage with Custom Settings

```python
# For more control over generation
image = pipeline(
    prompt="your prompt here",
    {"negative_prompt='low quality, worst quality'," if model_type == "sdxl" else ""}
    num_inference_steps=30,
    guidance_scale=8.0,
    {"width=1024," if model_type == "sdxl" else "width=512,"}
    {"height=1024," if model_type == "sdxl" else "height=512,"}
    generator=torch.Generator("cuda").manual_seed(42)
).images[0]
```

## Recommended Settings

{"### For Pony Diffusion Models:" if "pony" in base_model.lower() else ""}
{"- **Positive prompts**: Include quality tags like 'score_9, score_8_up, score_7_up'" if "pony" in base_model.lower() else ""}
{"- **Negative prompts**: Use 'score_4, score_5, score_6'" if "pony" in base_model.lower() else ""}
{"- **CFG Scale**: 6-8 works well" if "pony" in base_model.lower() else ""}

{"### For NoobAI Models:" if "noob" in base_model.lower() else ""}
{"- **Positive prompts**: Include 'masterpiece, best quality, very aesthetic'" if "noob" in base_model.lower() else ""}
{"- **Negative prompts**: Use 'lowres, bad anatomy, bad hands, text, error, missing fingers'" if "noob" in base_model.lower() else ""}
{"- **CFG Scale**: 5-7 recommended" if "noob" in base_model.lower() else ""}

- **Resolution**: {"1024x1024 or 832x1216 for portraits" if model_type == "sdxl" else "512x512 or 768x512 recommended"}
- **Steps**: 20-30 steps usually sufficient
- **Sampler**: Euler a, DPM++ 2M, or DPM++ SDE work well

## Model Details

This model was automatically converted from a SafeTensor checkpoint to the Diffusers format for easy use with the 🤗 Diffusers library.

### Technical Specifications
- **Architecture**: {"Stable Diffusion XL" if model_type == "sdxl" else "Stable Diffusion"}
- **Parameter Count**: {"~3.5B" if model_type == "sdxl" else "~860M"}
- **Precision**: Mixed precision (FP16/FP32)
- **VRAM Requirements**: {"~6GB" if model_type == "sdxl" else "~4GB"} (with FP16)

## License

This model is licensed under the CreativeML OpenRAIL-M license. Please ensure you comply with the license terms when using this model.

## Disclaimer

This model is converted from a community checkpoint. Please ensure you have the right to use and distribute the original model before using this converted version.
"""

        with open(os.path.join(output_dir, "README.md"), "w") as f:
            f.write(model_card_content)

    def upload_to_huggingface(self, model_dir: str, repo_name: str, token: str, private: bool = False):
        """
        Upload the converted model to Hugging Face
        """
        try:
            print(f"Creating repository: {repo_name}")

            # Create repository
            api = HfApi(token=token)
            create_repo(repo_name, token=token, private=private, exist_ok=True)

            print("Uploading model to Hugging Face...")
            api.upload_folder(
                folder_path=model_dir,
                repo_id=repo_name,
                token=token,
                commit_message="Upload converted Stable Diffusion model"
            )

            print(f"Model successfully uploaded to: https://huggingface.co/{repo_name}")

        except Exception as e:
            print(f"Error uploading to Hugging Face: {e}")
            raise e

# Main execution function with Colab parameter forms
def convert_and_upload_model():
    """
    Main function to convert SafeTensor to Diffusers and upload to Hugging Face
    Uses Google Colab parameter forms for easy input
    """

    # ========== GOOGLE COLAB PARAMETER FORMS ==========
    print("📋 Fill in the parameters below:")
    print("=" * 50)

    # Path to SafeTensor checkpoint file
    CHECKPOINT_PATH = "" # @param {type:"string"}

    # Hugging Face Token
    HF_TOKEN = "" # @param {type:"string"}

    # Repository name (format: username/model-name)
    REPO_NAME = "" # @param {type:"string"}

    # Base model selection
    BASE_MODEL_TYPE = "Auto-detect" # @param ["Auto-detect", "Stable Diffusion 1.5", "Stable Diffusion 2.1", "Stable Diffusion XL", "Pony Diffusion v6 XL", "NoobAI XL", "Custom"]

    # Custom base model (only used if "Custom" is selected above)
    CUSTOM_BASE_MODEL = "" # @param {type:"string"}

    # Repository privacy
    PRIVATE_REPO = True # @param {type:"boolean"}

    # Model name/title for the model card
    MODEL_NAME = "" # @param {type:"string"}

    # Model description
    MODEL_DESCRIPTION = "Converted from SafeTensor checkpoint to Diffusers format!" # @param {type:"string"}

    # ================================================

    converter = SafeTensorToDiffusersConverter()

    # Map base model selection to actual model paths
    base_model_mapping = {
        "Auto-detect": None,
        "Stable Diffusion 1.5": converter.supported_base_models['sd1.5'],
        "Stable Diffusion 2.1": converter.supported_base_models['sd2.1'],
        "Stable Diffusion XL": converter.supported_base_models['sdxl'],
        "Pony Diffusion v6 XL": converter.supported_base_models['pony'],
        "NoobAI XL": converter.supported_base_models['noobai'],
        "Custom": CUSTOM_BASE_MODEL if CUSTOM_BASE_MODEL else None
    }

    BASE_MODEL = base_model_mapping.get(BASE_MODEL_TYPE)

    # Validate inputs
    if not os.path.exists(CHECKPOINT_PATH):
        print(f"❌ Error: Checkpoint file not found at {CHECKPOINT_PATH}")
        print("Please upload your SafeTensor file to Google Colab and update the path.")
        return

    if not HF_TOKEN or HF_TOKEN.strip() == "":
        print("❌ Error: Please enter your Hugging Face token")
        print("Get your token from: https://huggingface.co/settings/tokens")
        return

    if not REPO_NAME or REPO_NAME.strip() == "":
        print("❌ Error: Please enter your repository name (format: username/model-name)")
        return

    if "/" not in REPO_NAME:
        print("❌ Error: Repository name must be in format 'username/model-name'")
        return

    try:
        # Create output directory
        output_dir = "./converted_model"

        print("\n🚀 Starting conversion process...")
        print(f"📁 Checkpoint: {CHECKPOINT_PATH}")
        print(f"🎯 Target repository: {REPO_NAME}")
        print(f"🔧 Base model type: {BASE_MODEL_TYPE}")
        print(f"🔒 Private repository: {PRIVATE_REPO}")
        print("-" * 50)

        # Convert SafeTensor to Diffusers
        converted_path = converter.convert_safetensor_to_diffusers(
            CHECKPOINT_PATH,
            output_dir,
            BASE_MODEL
        )

        # Update model card with custom information
        if MODEL_NAME or MODEL_DESCRIPTION:
            converter._create_detailed_model_card(
                output_dir,
                BASE_MODEL or "Auto-detected",
                CHECKPOINT_PATH,
                MODEL_NAME,
                MODEL_DESCRIPTION
            )

        # Upload to Hugging Face
        converter.upload_to_huggingface(
            converted_path,
            REPO_NAME,
            HF_TOKEN,
            PRIVATE_REPO
        )

        print("\n" + "="*50)
        print("🎉 CONVERSION COMPLETED SUCCESSFULLY!")
        print(f"🌐 Your model is now available at: https://huggingface.co/{REPO_NAME}")
        print("="*50)

        # Show usage example
        print(f"\n📖 Usage Example:")
        print(f"```python")
        print(f"from diffusers import StableDiffusionXLPipeline")
        print(f"import torch")
        print(f"")
        print(f"pipeline = StableDiffusionXLPipeline.from_pretrained(")
        print(f"    '{REPO_NAME}',")
        print(f"    torch_dtype=torch.float16")
        print(f")")
        print(f"pipeline = pipeline.to('cuda')")
        print(f"")
        print(f"prompt = 'a beautiful landscape'")
        print(f"image = pipeline(prompt).images[0]")
        print(f"image.save('generated_image.png')")
        print(f"```")

    except Exception as e:
        print(f"❌ Error during process: {e}")
        print("Please check your inputs and try again.")
        import traceback
        traceback.print_exc()

# Example usage with step-by-step instructions
def show_usage_example():
    """
    Show usage example and instructions
    """
    print("=" * 70)
    print("🎨 SAFETENSOR TO DIFFUSERS CONVERTER")
    print("=" * 70)
    print()
    print("📋 SUPPORTED BASE MODELS:")
    print("• Stable Diffusion 1.4/1.5")
    print("• Stable Diffusion 2.0/2.1")
    print("• Stable Diffusion XL")
    print("• Pony Diffusion v6 XL")
    print("• NoobAI XL")
    print("• Custom models (enter HuggingFace model ID)")
    print()
    print("🚀 INSTRUCTIONS:")
    print("1. Upload your SafeTensor (.safetensors) file to Google Colab")
    print("2. Get your HuggingFace token from: https://huggingface.co/settings/tokens")
    print("3. Fill in the parameter forms when you run convert_and_upload_model()")
    print("4. The script will automatically:")
    print("   • Detect the base model (or use your selection)")
    print("   • Convert to Diffusers format")
    print("   • Create a detailed model card")
    print("   • Upload to HuggingFace")
    print()
    print("💡 PARAMETER EXAMPLES:")
    print("CHECKPOINT_PATH: '/content/my-pony-model.safetensors'")
    print("HF_TOKEN: 'hf_your_actual_token_here'")
    print("REPO_NAME: 'myusername/my-pony-model-diffusers'")
    print("BASE_MODEL_TYPE: 'Pony Diffusion v6 XL' (or 'Auto-detect')")
    print()
    print("=" * 70)

# Show instructions when the cell is run
show_usage_example()

print("\n🔥 Ready to convert! Run the cell below to start:")
print("convert_and_upload_model()")

# Run the conversion (uncomment to execute)
convert_and_upload_model()